In [1]:
import pandas as pd
import numpy as np
#from openpyxl import load_workbook
from tqdm.notebook import tqdm
from typing import List
import os

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Following this code, https://github.com/pacocp/multiomic-fusion-NSCLC/blob/master/src/integration_late_fusion_all.ipynb

In [2]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from sklearn.model_selection import StratifiedShuffleSplit
import torch.nn.functional as F
import copy
import matplotlib.pyplot as plt
torch.manual_seed(0)

In [3]:
class model_together(nn.Module):
    def __init__(self, num_dtypes):
        super(model_together, self).__init__()
        self.weights = nn.Linear(num_dtypes, 3)
        #self.fc = nn.Linear(3, 1)

    def forward(self,xb):
        out = self.weights(xb)
        #out = self.fc(F.relu(out))
        out = torch.diagonal(out, 0, dim1=-2, dim2=-1)
        #out = F.softmax(out, dim=1)

        return out
def init_params(size, std=1.0): return (torch.rand(size)*std).requires_grad_()
def init_model(model, num_dtypes):
    with torch.no_grad():
        params = init_params((3,num_dtypes))
        params = params / params.sum(1, keepdim=True)
        model.weights.weight.copy_(params).float()
        model.weights.bias.fill_(0)

def norm_params(model):
    with torch.no_grad():
        # Push positive before scaling:
        #model.model.weight[model.model.weight < 0] = 0.
        #weights_norm = model.model.weight / model.model.weight.sum(1, keepdim=True)
        weights_norm = F.softmax(model.weights.weight, dim=1)
        model.weights.weight.copy_(weights_norm).float()
        model.weights.bias.fill_(0)

# this is ok to be with numpy but then need to be converted to tensors
def get_val_set(x, y, classes, percentage = 0.1):
    np.random.seed(42)
    x_train = np.array([]).reshape(0,x.shape[1])
    y_train = np.array([]).reshape(0,y.shape[1])
    x_val = np.array([]).reshape(0,x.shape[1])
    y_val = np.array([]).reshape(0,y.shape[1])
    for c in classes:
        indexes = np.where(y.argmax(axis=1) == c)[0]
        np.random.shuffle(indexes)
        len_val = int(percentage * len(indexes))
        len_train = len(indexes) - len_val
        index_train = indexes[0:len_train]
        index_val = indexes[len_train:]
        x_train = np.concatenate([x_train, x[index_train,...]], axis=0)
        y_train = np.concatenate([y_train, y[index_train]], axis=0)
        x_val = np.concatenate([x_val, x[index_val,...]], axis=0)
        y_val = np.concatenate([y_val, y[index_val]], axis=0)

    index_train = list(range(x_train.shape[0]))
    index_val = list(range(x_val.shape[0]))
    np.random.shuffle(index_train)
    np.random.shuffle(index_val)

    return x_train[index_train,...],y_train[index_train], x_val[index_val,...], y_val[index_val]

def train_model(model, dls, optimizer, criterion, num_epochs):
# copy code from the other notebook
    # Start training
    best_epoch = 0
    best_acc = 100
    val_preds = []
    val_labels = []
    val_labels_auc = []
    val_preds_auc = []
    train_preds = []
    train_labels = []
    train_preds_auc = []
    train_labels_auc = []
    best_model_wts = copy.deepcopy(model.state_dict())
    losses = {
        'train': [],
        'val': []
    }
    sizes = {'train': 0, 'val': 0}
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        batch_loss = 0
        sizes = {}
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            sizes[phase] = 0
            running_loss = 0
            running_corrects = 0
            for inputs, labels in dls[phase]:
                inputs = inputs.float()
                labels = labels.type(torch.LongTensor)
                #labels = F.one_hot(labels, num_classes=3)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    #import pdb; pdb.set_trace()
                    loss = criterion(outputs, labels)
                if phase == 'val':
                    val_preds += list(preds.numpy())
                    val_labels += list(labels.numpy())
                    val_labels_auc += [labels.numpy()]
                    val_preds_auc += [preds.numpy()]


                # Accumulating the loss over time
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)
                sizes[phase] += inputs.size(0)

                # backward + optimize only if in training phase
                if phase == 'train':
                    train_preds += list(preds.numpy())
                    train_labels += list(labels.numpy())
                    train_labels_auc += [labels.numpy()]
                    train_preds_auc += [preds.numpy()]
                    # Getting gradients w.r.t. parameters
                    loss.backward()
                    # Updating parameters
                    optimizer.step()
                    #norm_params(model)

            epoch_loss = running_loss / sizes[phase]
            epoch_acc = running_corrects.item() / sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc))
            losses[phase].append(epoch_loss)
            if phase == 'val' and epoch_loss < best_acc:
                best_epoch = epoch
                best_acc = epoch_loss
                #best_auc = epoch_auc
                best_model_wts = copy.deepcopy(model.state_dict())

        # Lr scheduler
        #lr_scheduler.step()

    print('Best epoch {}'.format(best_epoch))

    model.load_state_dict(best_model_wts)

    '''
    plt.figure()
    plt.plot(list(range(num_epochs)),losses['train'], label='train', color='blue')
    plt.plot(list(range(num_epochs)),losses['val'], label='val', color='orange')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    plt.close()
    '''
    norm_params(model)
    return model

def get_alphas_SGD(df, classes, data_types):
    x_train = []
    for _, row in tqdm(df.iterrows()):
        x = [[],[],[]]
        for d_type in data_types:
            if row['Has ' + d_type] != -1:
                luad_new = row[d_type+ ' Prob LUAD']
                hlt_new = row[d_type+ ' Prob HLT']
                lusc_new = row[d_type+ ' Prob LUSC']
                #luad_new = luad / (luad + hlt +lusc)
                #hlt_new = hlt / (luad + hlt +lusc)
                #lusc_new = lusc / (luad + hlt +lusc)
                x[0].append(luad_new)
                x[1].append(hlt_new)
                x[2].append(lusc_new)
            else:
                x[0].append(0)
                x[1].append(0)
                x[2].append(0)
        x_train.append(x)
    x_train = np.asarray(x_train)
    real = df['Real'].values

    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)

    for train_index, test_index  in sss.split(x_train, real):
        train_x, train_y = x_train[train_index,:,:], real[train_index]
        val_x, val_y = x_train[test_index,:,:], real[test_index]

    dl_train = DataLoader(list(zip(train_x,train_y)), batch_size=32, shuffle=True)
    dl_val = DataLoader(list(zip(val_x,val_y)), batch_size=32, shuffle=False)
    dls = {'train': dl_train, 'val': dl_val}

    model = model_together(len(data_types))
    init_model(model, len(data_types))

    optimizer = torch.optim.Adam(model.parameters(), 0.01)
    loss = nn.CrossEntropyLoss()

    model = train_model(model, dls, optimizer, loss, num_epochs=5)

    # a
    # once the optimization has been carried out
    weights = model.weights.weight.detach().numpy()

    alphas = {}
    index_dtype = 0
    for d_type in data_types:
        alphas[d_type] = []
        for i in range(len(classes)):
            alphas[d_type].append(weights[i,index_dtype])
        index_dtype += 1
    return alphas

In [48]:
import joblib

def integration_model(data_types: List[str], datasets: List[str], name: str, path1: str, path2: str,
                      fusion_type='probs', use_alphas=False, m_alphas=None) -> None:

    for d in datasets:
        csv_filename = path1 + 'data_integration_model_' + d + '_' + fusion_type + '_' + name + '.csv'

        # Data is read directly from CSV files
        data = {}
        for dt in data_types:
            file_name = f"{dt}_1.csv"
            data[dt] = pd.read_csv(path1 + dt + '/' + file_name)

        if use_alphas:
            print('Getting alphas from training set...')
            data_train = {}
            for dt in data_types:
                # Loading X_train and y_train separately
                with open(path2 + f'{dt}' + '/X_train.pkl', 'rb') as f:
                    X_train = joblib.load(f)
                with open(path2 + f'{dt}' + '/y_train.pkl', 'rb') as f:
                    y_train = joblib.load(f)

                print("X_train Length: ", len(X_train))
                print("y_train Length: ", len(y_train))

                # Ensure X_train is a DataFrame
                if not isinstance(X_train, pd.DataFrame):
                    X_train = pd.DataFrame(X_train, columns=[f'feature_{i}' for i in range(X_train.shape[1])])

                # Ensure y_train is a Series
                if not isinstance(y_train, pd.Series):
                    y_train = pd.Series(y_train, name='age_group')

                # Add y_train as a new column to X_train
                X_train['age_group'] = y_train.values
                data_train[dt] = X_train
            
            splits_alphas = {}
            for dt, df in data_train.items():
                alphas = get_alphas_SGD(df, [0,1,2], data_types)
                splits_alphas[dt] = alphas
                print(alphas)
            
            print('Saving alphas...')
            with open(path2 + 'data_integration_model_' + d + '_' + fusion_type + '_' + name + '_alphas.pkl', "wb") as f:
                joblib.dump(splits_alphas, f)


        #... [rest of the code is unchanged]


        for df_name, df in data.items():
            integration_probs = {
            'RNAseq': [],
            'microRNA': [],
            }
            integration_preds = []
            for _, row in tqdm(df.iterrows()):
                local_probs = {
                    'RNAseq': [],
                    'microRNA': [],
                }
                local_preds = []
                for d_type in data_types:
                    if row['Has ' + d_type] != -1:
                        rnaseq = row[d_type+ 'Prob RNAseq']
                        microRNA = row[d_type+ 'Prob microRNA']
                        rnaseq_new = rnaseq / (rnaseq + microRNA)
                        microRNA_new = microRNA / (rnaseq + microRNA)
                        local_probs['RNAseq'].append(rnaseq_new)
                        local_probs['microRNA'].append(microRNA_new)
                        local_preds.append(row[d_type + ' Pred'])
                    elif use_alphas:
                        local_probs['RNAseq'].append(0)
                        local_probs['microRNA'].append(0)

                if fusion_type == 'probs':
                    if use_alphas:
                        if m_alphas:
                        #alphas_manual = [0.65, 0.35]
                            alphas_manual = m_alphas
                            rnaseq_prob = integrate_probs(local_probs['RNAseq'], 0, alphas_manual)
                            microRNA_prob = integrate_probs(local_probs['microRNA'], 1, alphas_manual)
                        else:
                            rnaseq_prob_new = integrate_probs(local_probs['RNAseq'], 0, splits_alphas[df_name])
                            microRNA_prob_new = integrate_probs(local_probs['microRNA'], 1, splits_alphas[df_name])
                            rnaseq_prob = rnaseq_prob / (rnaseq_prob + microRNA_prob_new)
                            microRNA_prob = microRNA_prob / (rnaseq_prob + microRNA_prob)
                    else:
                        rnaseq_prob = integrate_probs(local_probs['RNAseq'])
                        microRNA_prob = integrate_probs(local_probs['microRNA'])
                    integration_probs['RNAseq'].append(rnaseq_prob)
                    integration_probs['microRNA'].append(microRNA_prob)

                    pred = np.argmax([rnaseq_prob,microRNA_prob], axis=0)
                    integration_preds.append(pred)

                elif fusion_type == 'preds':
                    if len(local_preds) == 2:
                        # if there are only two predictions, we need to fuse the probabilities
                        rnaseq_prob = integrate_probs(local_probs['RNAseq'])
                        microRNA_prob = integrate_probs(local_probs['microRNA'])

                        integration_probs['RNAseq'].append(rnaseq_prob)
                        integration_probs['microRNA'].append(microRNA_prob)

                        pred = np.argmax([rnaseq_prob,microRNA_prob], axis=0)

                    else:
                        pred = integrate_preds(local_preds)
                        if pred == 0:
                            rnaseq_prob = 1
                            microRNA_prob = 0
                        elif pred == 1:
                            rnaseq_prob = 0
                            hlt_prob = 1
                            microRNA_prob = 0
                        else:
                            rnaseq_prob = 0
                            microRNA_prob = 0

                        integration_probs['RNAseq'].append(rnaseq_prob)
                        integration_probs['microRNA'].append(rnaseq_prob)


                    integration_preds.append(pred)


            for cls in integration_probs.keys():
                df['Integration Prob '+ cls] = integration_probs[cls]

            df['Integration Pred'] = integration_preds

            # save to sheet
            output_file_path = path + 'data_integration_model_' + d + '_' + fusion_type + '_' + name + '_split_' + str(df_name) + '.csv'
            df.to_csv(output_file_path, index=False)

In [49]:
integration_model(data_types=["rnaseq", "microRNA"],
                  datasets=['test', 'train'], name="SGD-all_sources",
                  path1='data/', path2='baseline_model_outputs/', fusion_type='probs', use_alphas=True)

Getting alphas from training set...
X_train Length:  462
y_train Length:  462
X_train Length:  172
y_train Length:  172


0it [00:00, ?it/s]

KeyError: 'Has rnaseq'

In [33]:
with open('baseline_model_outputs/rnaseq/X_train.pkl', 'rb') as f:
    print(f.read(20))

b'\x80\x04\x95\xcf\x00\x00\x00\x00\x00\x00\x00\x8c\x13joblib.'


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, f1_score

data_model = pd.read_excel('../result_files/data_integration_model_test_probs_SGD-all_sources.xlsx',
              sheet_name=[0,1,2,3,4,5,6,7,8,9],engine='openpyxl')

accs = {
    'RNAseq': [],
    'miRNA': [],
    'Integration': {'RNAseq': [], 'miRNA': [], 'Integration': []}
}

f1_scores = {
    'RNAseq': [],
    'miRNA': [],
    'Integration': {'RNAseq': [], 'miRNA': [], 'Integration': []}
}

aucs = {
    'RNAseq': [],
    'microRNA': [],
    'Integration': {'RNAseq': [], 'miRNA': [], 'Integration': []}
}

auprcs = {
    'RNAseq': [],
    'microRNA': [],
    'Integration': {'RNAseq': [], 'miRNA': [], 'Integration': []}
}

for d_type in ['RNAseq', 'miRNA']:
        for df_name, df in data_model.items():
            if d_type != 'Integration':
                df_only = df.loc[df['Has '+ d_type] != -1]
            else:
                df_only = df
            probs = [[x,y,z] for x,y,z in zip(df_only[d_type + ' Prob RNAseq'], df_only[d_type + ' Prob microRNA'])]
            preds = df_only[d_type + ' Pred'].values
            probs = np.asarray(probs)
            real = df_only['Real'].values
            real_binarized = label_binarize(real, classes=[*range(3)])
            acc = accuracy_score(real, preds)*100
            f1 = f1_score(real, preds, average='weighted')*100
            auc = roc_auc_score(real, probs, multi_class='ovr')
            accs[d_type].append(acc)
            f1_scores[d_type].append(f1)
            aucs[d_type].append(auc)
            aucpr1 = average_precision_score(real_binarized[:, 0], probs[:, 0])
            aucpr2 = average_precision_score(real_binarized[:, 1], probs[:, 1])
            aucpr3 = average_precision_score(real_binarized[:, 2], probs[:, 2])
            auprcs[d_type].append(np.mean([aucpr1, aucpr2, aucpr3]))

            # integration
            probs_int = [[x,y,z] for x,y,z in zip(df_only['Integration Prob RNAseq'], df_only['Integration Prob microRNA'])]
            preds_int = df_only['Integration Pred'].values
            probs_int = np.asarray(probs_int)
            real = df_only['Real'].values
            real_binarized = label_binarize(real, classes=[*range(3)])
            acc_int = accuracy_score(real, preds_int)*100
            f1_int = f1_score(real, preds_int, average='weighted')*100
            auc_int = roc_auc_score(real, probs_int, multi_class='ovr')
            accs['Integration'][d_type].append(acc_int)
            f1_scores['Integration'][d_type].append(f1_int)
            aucs['Integration'][d_type].append(auc_int)
            aucpr1 = average_precision_score(real_binarized[:, 0], probs_int[:, 0])
            aucpr2 = average_precision_score(real_binarized[:, 1], probs_int[:, 1])
            aucpr3 = average_precision_score(real_binarized[:, 2], probs_int[:, 2])
            auprcs['Integration'][d_type].append(np.mean([aucpr1, aucpr2, aucpr3]))

In [ ]:
for d_type in ['RNAseq', 'microRNA']:
    print(d_type + ' ACC: {}+-{}'.format(np.mean(accs[d_type]),np.std(accs[d_type])))
    print(d_type + ' ACC: {}+-{}'.format(np.mean(accs['Integration'][d_type]),np.std(accs['Integration'][d_type])))
    print(5*'-')
print(10*'-')

for d_type in ['RNAseq', 'microRNA']:
    print(d_type + ' F1: {}+-{}'.format(np.mean(f1_scores[d_type]),np.std(f1_scores[d_type])))
    print(d_type + ' F1: {}+-{}'.format(np.mean(f1_scores['Integration'][d_type]),np.std(f1_scores['Integration'][d_type])))
    print(5*'-')
print(10*'-')

for d_type in ['RNAseq', 'microRNA']:
    print(d_type + ' AUC: {}+-{}'.format(np.mean(aucs[d_type]),np.std(aucs[d_type])))
    print(d_type + ' AUC: {}+-{}'.format(np.mean(aucs['Integration'][d_type]),np.std(aucs['Integration'][d_type])))
    print(5*'-')
print(10*'-')

for d_type in ['RNAseq', 'microRNA']:
    print(d_type + ' AUPRC: {}+-{}'.format(np.mean(auprcs[d_type]),np.std(auprcs[d_type])))
    print(d_type + ' AUPRC: {}+-{}'.format(np.mean(auprcs['Integration'][d_type]),np.std(auprcs['Integration'][d_type])))
    print(5*'-')

In [ ]:
import numpy as np
accs = []
f1_scores = []
aucs = []
auprcs = []
data_model = pd.read_excel('../result_files/data_integration_model_test_probs_SGD-all_sources.xlsx',
              sheet_name=[0,1,2,3,4,5,6,7,8,9],engine='openpyxl')

d_type = 'Integration'
for df_name, df in data_model.items():

    df_only = df
    probs = [[x,y,z] for x,y,z in zip(df_only[d_type + ' Prob RNAseq'], df_only[d_type + ' Prob microRNA'])]
    probs = np.asarray(probs)
    preds = df_only[d_type + ' Pred'].values

    real = df_only['Real'].values
    real_binarized = label_binarize(real, classes=[*range(3)])

    aucpr1 = average_precision_score(real_binarized[:, 0], probs[:, 0])
    aucpr2 = average_precision_score(real_binarized[:, 1], probs[:, 1])
    aucpr3 = average_precision_score(real_binarized[:, 2], probs[:, 2])

    acc = accuracy_score(real, preds)*100
    f1 = f1_score(real, preds, average='weighted')*100
    auc = roc_auc_score(real, probs, multi_class='ovr')
    accs.append(acc)
    f1_scores.append(f1)
    aucs.append(auc)
    auprcs.append(np.mean([aucpr1, aucpr2, aucpr3]))

print(d_type + ' ACC: {}+-{}'.format(np.mean(accs),np.std(accs)))
print(d_type + ' F1: {}+-{}'.format(np.mean(f1_scores),np.std(f1_scores)))
print(d_type + ' AUC: {}+-{}'.format(np.mean(aucs),np.std(aucs)))
print(d_type + ' AUPRC: {}+-{}'.format(np.mean(auprcs),np.std(auprcs)))

In [ ]:
for d_type in ['RNAseq', 'microRNA']:
    print(d_type + ' ACC: {}+-{}'.format(np.mean(accs[d_type]),np.std(accs[d_type])))
    print(d_type + ' ACC: {}+-{}'.format(np.mean(accs['Integration'][d_type]),np.std(accs['Integration'][d_type])))
    print(5*'-')
print(10*'-')

for d_type in ['RNAseq', 'microRNA']:
    print(d_type + ' F1: {}+-{}'.format(np.mean(f1_scores[d_type]),np.std(f1_scores[d_type])))
    print(d_type + ' F1: {}+-{}'.format(np.mean(f1_scores['Integration'][d_type]),np.std(f1_scores['Integration'][d_type])))
    print(5*'-')
print(10*'-')

for d_type in ['RNAseq', 'microRNA']:
    print(d_type + ' AUC: {}+-{}'.format(np.mean(aucs[d_type]),np.std(aucs[d_type])))
    print(d_type + ' AUC: {}+-{}'.format(np.mean(aucs['Integration'][d_type]),np.std(aucs['Integration'][d_type])))
    print(5*'-')

In [ ]:
os.mkdir('results_SGD/two-sources-integration')

In [ ]:
data_types = ['RNAseq', 'microRNA']

i = 0
j = 1

for d_type1 in data_types[i:]:
    for d_type2 in data_types[j:]:
        name = d_type1 + '-' + d_type2
        integration_model(data_types=[d_type1, d_type2],
                          datasets=['test'], name=name,
                          path='results_SGD/two-sources-integration/',
                          fusion_type='probs', use_alphas=True)
    i += 1
    j = i + 1

In [ ]:
from glob import glob
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, f1_score, average_precision_score
from sklearn.preprocessing import label_binarize

data_types = ['RNAseq', 'microRNA']

accs = {}
f1_scores = {}
aucs = {}
aucprcs = {}

i = 0
j = 1

for d_type1 in data_types[i:]:
    for d_type2 in data_types[j:]:
        name = d_type1 + '-' + d_type2
        data_model = pd.read_excel('../result_files/two-sources-integration/data_integration_model_test_probs_'+name+'.xlsx',
              sheet_name=[0,1,2,3,4,5,6,7,8,9],engine='openpyxl')

        accs[name] = {}
        f1_scores[name] = {}
        aucs[name] = {}
        aucprcs[name] = {}

        accs[name]['Integration'] = []
        accs[name][d_type1+'Int'] = []
        accs[name][d_type2+'Int'] = []

        f1_scores[name]['Integration'] = []
        f1_scores[name][d_type1+'Int'] = []
        f1_scores[name][d_type2+'Int'] = []

        aucs[name]['Integration'] = []
        aucs[name][d_type1+'Int'] = []
        aucs[name][d_type2+'Int'] = []

        aucprcs[name]['Integration'] = []
        aucprcs[name][d_type1+'Int'] = []
        aucprcs[name][d_type2+'Int'] = []

        for df_name, df in data_model.items():
            # take those where the two sources has data
            df_only = df.loc[(df['Has '+ d_type1] != -1) | (df['Has ' + d_type2] != -1)]
            df_dt1 = df.loc[df['Has '+ d_type1] != -1]
            df_dt2 = df.loc[df['Has '+ d_type2] != -1]
            probs = [[x,y,z] for x,y,z in zip(df_only['Integration Prob RNAseq'], df_only['Integration Prob microRNA'])]
            probs = np.asarray(probs)
            preds = df_only['Integration Pred'].values
            real = df_only['Real'].values
            acc = accuracy_score(real, preds)*100
            f1 = f1_score(real, preds, average='weighted')*100
            auc = roc_auc_score(real, probs, multi_class='ovr')
            real_binarized = label_binarize(real, classes=[*range(3)])

            aucpr1 = average_precision_score(real_binarized[:, 0], probs[:, 0])
            aucpr2 = average_precision_score(real_binarized[:, 1], probs[:, 1])
            aucpr3 = average_precision_score(real_binarized[:, 2], probs[:, 2])

            accs[name]['Integration'].append(acc)
            f1_scores[name]['Integration'].append(f1)
            aucs[name]['Integration'].append(auc)
            aucprcs[name]['Integration'].append([aucpr1,aucpr2,aucpr3])

            # dtype1
            probs = [[x,y,z] for x,y,z in zip(df_dt1['Integration Prob RNAseq'], df_dt1['Integration Prob microRNA'])]
            probs = np.asarray(probs)
            preds = df_dt1['Integration Pred'].values
            real = df_dt1['Real'].values
            real_binarized = label_binarize(real, classes=[*range(3)])
            acc = accuracy_score(real, preds)*100
            f1 = f1_score(real, preds, average='weighted')*100
            auc = roc_auc_score(real, probs, multi_class='ovr')

            aucpr1 = average_precision_score(real_binarized[:, 0], probs[:, 0])
            aucpr2 = average_precision_score(real_binarized[:, 1], probs[:, 1])
            aucpr3 = average_precision_score(real_binarized[:, 2], probs[:, 2])

            accs[name][d_type1+'Int'].append(acc)
            f1_scores[name][d_type1+'Int'].append(f1)
            aucs[name][d_type1+'Int'].append(auc)
            aucprcs[name][d_type1+'Int'].append(np.mean([aucpr1,aucpr2,aucpr3]))

            # dtype2
            probs = [[x,y,z] for x,y,z in zip(df_dt2['Integration Prob RNAseq'], df_dt1['Integration Prob microRNA'])]
            preds = df_dt2['Integration Pred'].values
            probs = np.asarray(probs)
            real = df_dt2['Real'].values
            real_binarized = label_binarize(real, classes=[*range(3)])
            acc = accuracy_score(real, preds)*100
            f1 = f1_score(real, preds, average='weighted')*100
            auc = roc_auc_score(real, probs, multi_class='ovr')
            aucpr1 = average_precision_score(real_binarized[:, 0], probs[:, 0])
            aucpr2 = average_precision_score(real_binarized[:, 1], probs[:, 1])
            aucpr3 = average_precision_score(real_binarized[:, 2], probs[:, 2])

            accs[name][d_type2+'Int'].append(acc)
            f1_scores[name][d_type2+'Int'].append(f1)
            aucs[name][d_type2+'Int'].append(auc)
            aucprcs[name][d_type2+'Int'].append(np.mean([aucpr1,aucpr2,aucpr3]))

    i += 1
    j = i + 1

In [ ]:
i = 0
j = 1

for d_type1 in data_types[i:]:
    for d_type2 in data_types[j:]:
        print(5*'-')
        name = d_type1 + '-' + d_type2
        print(name + ' ACC: {}+-{}'.format(np.mean(accs[name]['Integration']),np.std(accs[name]['Integration'])))
        print(d_type1+'Int' + ' ACC: {}+-{}'.format(np.mean(accs[name][d_type1+'Int']),np.std(accs[name][d_type1+'Int'])))
        print(d_type2+'Int' + ' ACC: {}+-{}'.format(np.mean(accs[name][d_type2+'Int']),np.std(accs[name][d_type2+'Int'])))

    i += 1
    j = i + 1
print(10*'-')

i = 0
j = 1

for d_type1 in data_types[i:]:
    for d_type2 in data_types[j:]:
        name = d_type1 + '-' + d_type2
        print(5*'-')
        print(name + ' F1: {}+-{}'.format(np.mean(f1_scores[name]['Integration']),np.std(f1_scores[name]['Integration'])))
        print(d_type1+'Int' + ' F1: {}+-{}'.format(np.mean(f1_scores[name][d_type1+'Int']),np.std(f1_scores[name][d_type1+'Int'])))
        print(d_type2+'Int' + ' F1: {}+-{}'.format(np.mean(f1_scores[name][d_type2+'Int']),np.std(f1_scores[name][d_type2+'Int'])))
    i += 1
    j = i + 1
print(10*'-')

i = 0
j = 1

for d_type1 in data_types[i:]:
    for d_type2 in data_types[j:]:
        print(5*'-')
        name = d_type1 + '-' + d_type2
        print(name + ' AUC: {}+-{}'.format(np.mean(aucs[name]['Integration']),np.std(aucs[name]['Integration'])))
        print(d_type1+'Int' + ' AUC: {}+-{}'.format(np.mean(aucs[name][d_type1+'Int']),np.std(aucs[name][d_type1+'Int'])))
        print(d_type2+'Int' + ' AUC: {}+-{}'.format(np.mean(aucs[name][d_type2+'Int']),np.std(aucs[name][d_type2+'Int'])))
    i += 1
    j = i + 1

i = 0
j = 1
for d_type1 in data_types[i:]:
    for d_type2 in data_types[j:]:
        print(5*'-')
        name = d_type1 + '-' + d_type2
        print(name + ' AUPRC: {}+-{}'.format(np.mean(aucprcs[name]['Integration']),np.std(aucprcs[name]['Integration'])))
        print(d_type1+'Int' + ' AUPRC: {}+-{}'.format(np.mean(aucprcs[name][d_type1+'Int']),np.std(aucprcs[name][d_type1+'Int'])))
        print(d_type2+'Int' + ' AUPRC: {}+-{}'.format(np.mean(aucprcs[name][d_type2+'Int']),np.std(aucprcs[name][d_type2+'Int'])))
    i += 1
    j = i + 1